In [1]:
import pandas as pd
import numpy as np

In [2]:
movie = pd.read_csv('../dataset/movie.csv')
rating = pd.read_csv('../dataset/rating.csv')
rating = rating.drop('timestamp', axis=1)

In [3]:
movie.describe(), movie['movieId'].unique().shape

(             movieId
 count   27278.000000
 mean    59855.480570
 std     44429.314697
 min         1.000000
 25%      6931.250000
 50%     68068.000000
 75%    100293.250000
 max    131262.000000,
 (27278,))

In [4]:
rating.keys(), rating.describe()


(Index(['userId', 'movieId', 'rating'], dtype='object'),
              userId       movieId        rating
 count  2.000026e+07  2.000026e+07  2.000026e+07
 mean   6.904587e+04  9.041567e+03  3.525529e+00
 std    4.003863e+04  1.978948e+04  1.051989e+00
 min    1.000000e+00  1.000000e+00  5.000000e-01
 25%    3.439500e+04  9.020000e+02  3.000000e+00
 50%    6.914100e+04  2.167000e+03  3.500000e+00
 75%    1.036370e+05  4.770000e+03  4.000000e+00
 max    1.384930e+05  1.312620e+05  5.000000e+00)

In [5]:
rating_small = rating[rating['userId'] <= 100000]
rating_small.describe()


,userId,movieId,rating
count,1.447209e+07,1.447209e+07,1.447209e+07
mean,4.987456e+04,9.032079e+03,3.525358e+00
std,2.896016e+04,1.975944e+04,1.052196e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,2.469100e+04,9.030000e+02,3.000000e+00
50%,4.994400e+04,2.174000e+03,3.500000e+00
75%,7.505100e+04,4.776000e+03,4.000000e+00
max,1.000000e+05,1.312600e+05,5.000000e+00


In [6]:
movies_present= rating_small['movieId'].unique()
movies_present.shape
movie_small = movie[movie['movieId'].isin(movies_present)]

In [7]:
movie_small.describe()

,movieId
count,24829.000000
mean,55315.805308
std,43557.260812
min,1.000000
25%,6306.000000
50%,58367.000000
75%,95521.000000
max,131260.000000


In [8]:
df = rating_small.merge(movie_small, on='movieId')
df = df.dropna()
df.drop_duplicates(inplace=True)

In [9]:
df.describe(), df.shape


(             userId       movieId        rating
 count  1.447209e+07  1.447209e+07  1.447209e+07
 mean   4.987456e+04  9.032079e+03  3.525358e+00
 std    2.896016e+04  1.975944e+04  1.052196e+00
 min    1.000000e+00  1.000000e+00  5.000000e-01
 25%    2.469100e+04  9.030000e+02  3.000000e+00
 50%    4.994400e+04  2.174000e+03  3.500000e+00
 75%    7.505100e+04  4.776000e+03  4.000000e+00
 max    1.000000e+05  1.312600e+05  5.000000e+00,
 (14472091, 5))

In [10]:
col1 = 'movieId'
col2 = 'title'
second_2 = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().sort_values(ascending=False)
one_instance_to_be_removed=second_2[second_2 > 1]
dic ={i:[] for i in one_instance_to_be_removed.index}
for i,row in df.iterrows():
    if row[col2] in dic and row[col1] not in dic[row[col2]]:
        dic[row[col2]].append(row[col1])

dic

{'Hamlet (2000)': [3598, 65665],
 'Blackout (2007)': [66140, 85070],
 'Chaos (2005)': [47254, 67459],
 'Emma (1996)': [838, 26958],
 'Casanova (2005)': [42015, 128862],
 '20,000 Leagues Under the Sea (1997)': [102190, 114130],
 'Aladdin (1992)': [588, 114240],
 'War of the Worlds (2005)': [34048, 64997],
 'Clear History (2013)': [104155, 122940],
 'Darling (2007)': [130062, 93279],
 'Beneath (2013)': [104035, 115777],
 'Paradise (2013)': [113459, 121586],
 'Men with Guns (1997)': [1788, 26982],
 'Offside (2006)': [48682, 80330]}

In [11]:
list = [dic[i][-1] for i in dic.keys()]
df = df[~df['movieId'].isin(list)]
df.describe(), df.shape


(             userId       movieId        rating
 count  1.447169e+07  1.447169e+07  1.447169e+07
 mean   4.987455e+04  9.030551e+03  3.525370e+00
 std    2.896017e+04  1.975743e+04  1.052195e+00
 min    1.000000e+00  1.000000e+00  5.000000e-01
 25%    2.469100e+04  9.030000e+02  3.000000e+00
 50%    4.994300e+04  2.174000e+03  3.500000e+00
 75%    7.505100e+04  4.776000e+03  4.000000e+00
 max    1.000000e+05  1.312600e+05  5.000000e+00,
 (14471692, 5))

In [12]:
movie_counts = pd.DataFrame(df["title"].value_counts())
movie_counts.shape,movie_counts.head()

((24815, 1),
                                   title
 Pulp Fiction (1994)               48619
 Forrest Gump (1994)               47756
 Shawshank Redemption, The (1994)  45777
 Silence of the Lambs, The (1991)  45653
 Jurassic Park (1993)              43177)

In [13]:
rare_movies = movie_counts[movie_counts["title"] <= 50].index
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.shape, rare_movies.shape

((14324593, 5), (15282,))

In [14]:
# user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
# user_movie_df.shape
common_movies.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [15]:
common_movies.to_csv('processed_data.csv', index=False)

: 